## Milky Way

In [ ]:
# Third-party
from astropy.constants import G
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from scipy.optimize import leastsq

# Custom
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

In [ ]:
# load tables of mass enclosed / circular velocity measurements
gnedin_tbl = np.genfromtxt("data/gnedin_mwmass_tbl.txt",
                           delimiter=',', names=True, dtype=None)
kuepper_tbl = np.genfromtxt("data/kuepper15_mwmass_tbl.txt",
                            delimiter=',', names=True, dtype=None)

In [ ]:
Menc1 = gnedin_tbl['Menc']
Menc_l = gnedin_tbl['neg_err']
Menc_u = gnedin_tbl['pos_err']
Menc_err1 = np.max([Menc_u, Menc_l], axis=0) # HACK: take maximum uncertainty

# Andreas compiled circular velocity measurements so need to turn into mass
kuepper_tbl = kuepper_tbl[kuepper_tbl['source'].astype(str) != 'Gibbons et al. (2014)']
Menc2 = (kuepper_tbl['radius']*u.kpc * (kuepper_tbl['v_c']*u.km/u.s)**2 / G).to(u.Msun).value
Menc_l = Menc2 - (kuepper_tbl['radius']*u.kpc * ((kuepper_tbl['v_c']-kuepper_tbl['neg_err'])*u.km/u.s)**2 / G).to(u.Msun).value
Menc_u = (kuepper_tbl['radius']*u.kpc * ((kuepper_tbl['v_c']+kuepper_tbl['pos_err'])*u.km/u.s)**2 / G).to(u.Msun).value - Menc2
Menc_err2 = np.max([Menc_u, Menc_l], axis=0) # HACK: take maximum uncertainty

In [ ]:
# compile all mass measurements
r = np.concatenate((gnedin_tbl['radius'], kuepper_tbl['radius']))
Menc = np.concatenate((Menc1, Menc2))
Menc_err = np.concatenate((Menc_err1, Menc_err2))

In [ ]:
# sort by radius and remove measurements at very small radii
idx = r.argsort()
obs_Menc = Menc[idx][2:]
obs_Menc_err = Menc_err[idx][2:]
obs_r = r[idx][2:]

In [ ]:
def get_potential(log_M_h, log_r_s, log_M_n, log_a):
    mw_potential = gp.CCompositePotential()
    mw_potential['bulge'] = gp.HernquistPotential(m=5E9, c=1., units=galactic)
    mw_potential['disk'] = gp.MiyamotoNagaiPotential(m=6.8E10*u.Msun, a=3*u.kpc, b=280*u.pc,
                                                     units=galactic)
    mw_potential['nucl'] = gp.HernquistPotential(m=np.exp(log_M_n), c=np.exp(log_a)*u.pc,
                                                 units=galactic)
    mw_potential['halo'] = gp.NFWPotential(m=np.exp(log_M_h), r_s=np.exp(log_r_s), units=galactic)

    return mw_potential

In [ ]:
# Initial guess for the parameters:
x0 = [np.log(6E11), np.log(20.), np.log(2E9), np.log(100.)] # a is in pc
init_pot = get_potential(*x0)

xyz = np.zeros((3, obs_r.size))
xyz[0] = obs_r

In [ ]:
def f(p):
    pot = get_potential(*p)
    model_menc = pot.mass_enclosed(xyz).to(u.Msun).value
    return (model_menc - obs_Menc) / obs_Menc_err

In [ ]:
p_opt, ier = leastsq(f, x0=x0)
assert ier in range(1,4+1)
fit_potential = get_potential(*p_opt)

In [ ]:
fig = fit_potential.plot_contours(grid=(np.linspace(0,20,128), 0, np.linspace(-5,5,64)))
fig.axes[0].set_aspect('equal')

In [ ]:
fit_potential.density([20,0,0.])

In [ ]:
fig = fit_potential.plot_density_contours(grid=(np.linspace(0,20,128), 0, np.linspace(-5,5,64)), filled=False,
                                          levels=np.logspace(5, 10., 24), colors='k', cmap=None)
fig.axes[0].set_aspect('equal')

In [ ]:
# plot initial guess, fit profile:
r = np.logspace(-3.5, 2.6, 1024)
xyz = np.zeros((3,r.size))
xyz[0] = r
fit_menc = fit_potential.mass_enclosed(xyz*u.kpc)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,6))

ax.errorbar(obs_r, obs_Menc, yerr=obs_Menc_err, marker='o', markersize=4,
            color='k', alpha=1., ecolor='#aaaaaa', capthick=0,
            linestyle='none', elinewidth=1.)

ax.loglog(r, fit_menc.value, marker='', color="#3182bd",
          linewidth=2, alpha=0.7)

ax.set_xlim(1E-3, 10**2.6)
ax.set_ylim(7E6, 10**12.25)

ax.set_xlabel('$r$ [kpc]')
ax.set_ylabel('$M(<r)$ [M$_\odot$]')

ax.set_xscale('log')
ax.set_yscale('log')

fig.tight_layout()